In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf, col,mean
from pyspark.sql.types import *
import numpy as np
from sklearn.datasets import make_moons, make_blobs
import pandas as pd



In [2]:
spark = SparkSession.builder.appName('test-aggregate-event-profile-score').getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

In [3]:
event_alert_score_sdf = spark.read.json("/Users/tuhinsharma/Documents/sstech/demo/hive_data/wgtraffic_event_alert_score.json").persist()
profile_score_sdf = spark.read.json("/Users/tuhinsharma/Documents/sstech/demo/hive_data/wgtraffic_profile_score.json").persist()


In [4]:
event_alert_score_sdf.show(2)

+-----------+------+---+-------------+-----------+--------------------+---+--------------+-------------+--------------------+----+
|alert_score|alerts|  d|  entity_name|event_score|                guid|  m|   source_type|    timestamp|           user_name|   y|
+-----------+------+---+-------------+-----------+--------------------+---+--------------+-------------+--------------------+----+
|        0.0|      | 25|192.168.0.111|      100.0|d679d707-1bb8-407...|  4|demo_wgtraffic|1556206508000|madison.prince@ss...|2019|
|        0.0|      | 25| 192.168.0.86|      100.0|cf7770b3-26d2-462...|  4|demo_wgtraffic|1556206349000|flash.hunt@sstech.us|2019|
+-----------+------+---+-------------+-----------+--------------------+---+--------------+-------------+--------------------+----+
only showing top 2 rows



In [5]:
profile_score_sdf.show(2)

+------------------+------------------+-------------------+------------------+------------+-----------+----------------+----+
|     PAS_isolation|        PAS_kmeans|            PAS_svm|     PAS_wgtraffic|        name|time_window|       timestamp|type|
+------------------+------------------+-------------------+------------------+------------+-----------+----------------+----+
|  31.7455293399957|               0.0|0.03216223111822964|10.592563857037975| 192.168.0.1|       hour|1556226097561538|  ip|
|28.636263100254496|1.0236024856567383|0.04569207506500132| 9.901852553658745|192.168.0.14|       hour|1556226097561538|  ip|
+------------------+------------------+-------------------+------------------+------------+-----------+----------------+----+
only showing top 2 rows



In [6]:
TIME_WINDOW_LIST = ["hour", "day", "week"]
USER_ENTITY_TYPE = "user"
IP_ENTITY_TYPE = "ip"
dic = {"hour":60*60*1000000,"day":24*60*60*1000000,"week":7*24*60*60*1000000}

In [7]:
def calculate_final_anomaly_score(profile_sdf,event_sdf,time_window,actor_type):
    profile_sdf.createOrReplaceTempView("profile_sdf")
    event_sdf.createOrReplaceTempView("event_sdf")
    timestamp_sdf = spark.sql("select min(timestamp) as timestamp from profile_sdf where time_window='"+time_window+"' and type='"+actor_type+"'")
    timestamp = timestamp_sdf.rdd.map(lambda x:x["timestamp"]).collect()[0]
    start_timestamp = str(int((timestamp - dic[time_window])/1000))
    end_timestamp = str(int(timestamp/1000))
    profile_type_time_window_sdf = spark.sql("select type ,timestamp, name , time_window , PAS_wgtraffic from profile_sdf where time_window='"+time_window+"' and type='"+actor_type+"' and timestamp="+str(timestamp))
    profile_type_time_window_sdf.show(2)
    print(profile_type_time_window_sdf.count())
    profile_type_time_window_sdf.select("time_window").distinct().show()

    event_time_window_sdf = spark.sql("select timestamp,entity_name ,user_name,event_score from event_sdf where timestamp > "+start_timestamp+" and timestamp < "+end_timestamp)
    event_time_window_sdf.createOrReplaceTempView("event_time_window_sdf")
    if actor_type == IP_ENTITY_TYPE:
        event_type_time_window_sdf = spark.sql("select entity_name as name,mean(event_score) as EAS_wgtraffic from event_time_window_sdf group by entity_name")
    else:
        event_type_time_window_sdf = spark.sql("select user_name as name,mean(event_score) as EAS_wgtraffic from event_time_window_sdf group by user_name")


    event_type_time_window_sdf.show(3)
    print(event_type_time_window_sdf.count())
    
    joined_sdf = profile_type_time_window_sdf.join(event_type_time_window_sdf,on="name",how="left_outer")
    joined_sdf = joined_sdf.fillna(0)
    joined_sdf.show(4)


    
    from pyspark.sql.functions import udf, array
    from pyspark.sql.types import DoubleType

    avg_cols = udf(lambda array: sum(array)/len(array), FloatType())

    result_sdf = joined_sdf.withColumn("AS_wgtraffic", avg_cols(array("PAS_wgtraffic", "EAS_wgtraffic")))
    result_sdf.show(10)
    print(result_sdf.count())
    return result_sdf

    

In [8]:
result_sdf = calculate_final_anomaly_score(profile_score_sdf,event_alert_score_sdf,"week","user")

+----+----------------+--------------------+-----------+------------------+
|type|       timestamp|                name|time_window|     PAS_wgtraffic|
+----+----------------+--------------------+-----------+------------------+
|user|1556226350347841|camille.gordon@ss...|       week|0.5694417203772256|
|user|1556226350347841|james.prince@sste...|       week| 4.857787961136455|
+----+----------------+--------------------+-----------+------------------+
only showing top 2 rows

117
+-----------+
|time_window|
+-----------+
|       week|
+-----------+

+--------------------+-----------------+
|                name|    EAS_wgtraffic|
+--------------------+-----------------+
|madison.cook@sste...|98.89325924555384|
|clark.kent@sstech.us|            100.0|
|JohnWhiteLateralM...|99.47813015769046|
+--------------------+-----------------+
only showing top 3 rows

114
+--------------------+----+----------------+-----------+------------------+-----------------+
|                name|type|       